### C++ Environment trials

In [ ]:
%%file trial.cpp

#include <iostream>
#include <Eigen/Dense>
#include <cmath>

Eigen::MatrixXd distance(Eigen::MatrixXd xs, Eigen::MatrixXd centroid)
{    
    /*
    Computes matrix of squared distance from each point to each centroid.
    
    Parameters:
    -----------
    xs : ndarray of n points in d dimensional Euclidean space (nxd)
    centroid: ndarray of k centroids in d dimensional Euclidean space (kxd)
    
    Returns:
    --------
    distance: matrix of squared distances (nxk)
    */
    int n = xs.rows();
    int k = centroid.rows();
    int d = xs.cols();
    Eigen::MatrixXd distance_matrix(n,k);
    for (int col_index=0; col_index<k; col_index++)
    {
        for (int row_index=0; row_index<n; row_index++)
        {
            double dist = (xs.row(row_index) - centroid.row(col_index)).norm();
            distance_matrix(row_index,col_index) = std::pow(dist,2);
        }
    }
    return distance_matrix;
}

double cost(Eigen::MatrixXd distance_matrix)
{
    /*
    Computes the cost of a set of points with respect to a collection of centroids
    
    Parameters:
    -----------
    distance_matrix : matrix of squared distances (nxk); likely returned from distance() function
    
    Returns:
    --------
    cost: cost with respect to centroids
    */
    double cost;
    cost = distance_matrix.rowwise().minCoeff().sum();
    return cost;
}

Eigen::VectorXd centroid_weights(Eigen::MatrixXd distance_matrix)
{
    /*
    Computes weights as defined in step 7 of the k-means|| algorithm
        
    Parameters:
    -----------
    distance_matrix : matrix of squared distances (nxk); likely returned from distance() function
    centroid: ndarray of k centroids in d dimensional Euclidean space (kxd)
    
    Returns:
    --------
    w_x: ndarray of weights applied to centroids (kx1)
    */
    int n = distance_matrix.rows();
    int k = distance_matrix.cols();
    Eigen::MatrixXd weight_matrix(n,k);
    weight_matrix.fill(0);
    Eigen::VectorXd w_x(k);
    for (int row_index=0; row_index<n; row_index++)
    {
        double row_minimum_value = distance_matrix.row(row_index).minCoeff();
        for (int col_index=0; col_index<k; col_index++)
        {
            if (distance_matrix(row_index,col_index)==row_minimum_value)
            {
                weight_matrix(row_index,col_index)=1;
            }
        }
    }
    w_x = weight_matrix.colwise().sum();
    return w_x;
}

int main()
{
    Eigen::MatrixXd A1(3,3);
    A1 << 5 ,7, 9,
          4, 5, 6,
          7, 8, 9;
    Eigen::MatrixXd A2 = Eigen::MatrixXd::Identity(3, 3);
    Eigen::MatrixXd A3(1,3);
    A3 << 5 ,7, 9;
    Eigen::MatrixXd A4(3,3);
    A4.fill(0);
    
    
    std::cout << centroid_weights(A1,A3) << std::endl;    
    
}

In [ ]:
%%bash
g++ trial.cpp -o trial.exe -std=c++14 -I/usr/include/eigen3

In [ ]:
%%bash
./trial.exe

### Wrapping (3 helper functions)

In [1]:
%%file helper_wrap.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
cfg['include_dirs'] = ['../notebooks/eigen3']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#include <pybind11/eigen.h>
#include <pybind11/stl.h>
#include <pybind11/cast.h>

Eigen::MatrixXd distance(Eigen::MatrixXd xs, Eigen::MatrixXd centroid)
{    
    /*
    Computes matrix of squared distance from each point to each centroid.
    
    Parameters:
    -----------
    xs : ndarray of n points in d dimensional Euclidean space (nxd)
    centroid: ndarray of k centroids in d dimensional Euclidean space (kxd)
    
    Returns:
    --------
    distance: matrix of squared distances (nxk)
    */
    int n = xs.rows();
    int k = centroid.rows();
    int d = xs.cols();
    Eigen::MatrixXd distance_matrix(n,k);
    for (int col_index=0; col_index<k; col_index++)
    {
        for (int row_index=0; row_index<n; row_index++)
        {
            double dist = (xs.row(row_index) - centroid.row(col_index)).norm();
            distance_matrix(row_index,col_index) = std::pow(dist,2);
        }
    }
    return distance_matrix;
}

double cost(Eigen::MatrixXd distance_matrix)
{
    /*
    Computes the cost of a set of points with respect to a collection of centroids
    
    Parameters:
    -----------
    distance_matrix : matrix of squared distances (nxk); likely returned from distance() function
    
    Returns:
    --------
    cost: cost with respect to centroids
    */
    double cost;
    cost = distance_matrix.rowwise().minCoeff().sum();
    return cost;
}

Eigen::VectorXd centroid_weights(Eigen::MatrixXd distance_matrix)
{
    /*
    Computes weights as defined in step 7 of the k-means|| algorithm
        
    Parameters:
    -----------
    distance_matrix : matrix of squared distances (nxk); likely returned from distance() function
    centroid: ndarray of k centroids in d dimensional Euclidean space (kxd)
    
    Returns:
    --------
    w_x: ndarray of weights applied to centroids (kx1)
    */
    int n = distance_matrix.rows();
    int k = distance_matrix.cols();
    Eigen::MatrixXd weight_matrix(n,k);
    weight_matrix.fill(0);
    Eigen::VectorXd w_x(k);
    for (int row_index=0; row_index<n; row_index++)
    {
        double row_minimum_value = distance_matrix.row(row_index).minCoeff();
        for (int col_index=0; col_index<k; col_index++)
        {
            if (distance_matrix(row_index,col_index)==row_minimum_value)
            {
                weight_matrix(row_index,col_index)=1;
            }
        }
    }
    w_x = weight_matrix.colwise().sum();
    return w_x;
}

PYBIND11_MODULE(helper_wrap, m) {
    m.doc() = "helpers functions for k-means|| initialization";
    m.def("distance", &distance, "distance matrix function");
    m.def("cost", &cost, "cost function");
    m.def("centroid_weights", &centroid_weights, "distance matrix function");
}


Writing helper_wrap.cpp


In [2]:
import cppimport
cppimport.force_rebuild()
helper_wrap = cppimport.imp("helper_wrap")

### Testing the wrapped functions in python

In [3]:
import numpy as np

xs = np.matrix([
    [1,2,3],
    [4,5,6],
    [7,8,9]
])
centroid = np.matrix([
    [1,1,1],
    [2,2,2]
])

In [4]:
distance_matrix = helper_wrap.distance(xs,centroid)
distance_matrix

array([[  5.,   2.],
       [ 50.,  29.],
       [149., 110.]])

In [5]:
helper_wrap.cost(distance_matrix)

141.0

In [7]:
helper_wrap.centroid_weights(distance_matrix)

array([0., 3.])

### Scratch paper and previous work

In [ ]:
%%file wrap.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
cfg['include_dirs'] = ['../notebooks/eigen3']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#include <pybind11/eigen.h>
#include <pybind11/stl.h>
#include <pybind11/cast.h>


namespace py = pybind11;
double min_dist(Eigen::VectorXd x, std::list<Eigen::VectorXd> C)
{
    double min_distance = std::numeric_limits<double>::infinity();
    for (Eigen::VectorXd centroid: C)
    {
        double distance = (x-centroid).norm();
        if (distance < min_distance)
        {
            min_distance = distance;
        }
    }
    return min_distance;
}

PYBIND11_MODULE(wrap, m) {
    m.doc() = "pybind11 example plugin";
    m.def("min_dist", &min_dist, "A vectroized square function.");
}

In [ ]:
import cppimport
cppimport.force_rebuild()
wrap = cppimport.imp("wrap")

In [ ]:
import numpy as np

x = np.array([1,2,3])
c1 = np.array([1,2,2.5])
c2 = np.array([0,0,0])
c3 = np.array([1,1,1])
C = list([c1, c2, c3])



wrap.min_dist(x,C)

In [ ]:
%%file junk.cpp

#include <iostream>
#include <list>
#include <Eigen/Dense>
#include <cmath>
#include <limits>

double min_dist(Eigen::VectorXd x, std::list<Eigen::VectorXd> C)
{
    double min_distance = std::numeric_limits<double>::infinity();
    
    for (Eigen::VectorXd centroid: C)
    {
        double distance = (x-centroid).norm();
        
        if (distance < min_distance)
        {
            min_distance = distance;
        }
    }
    
    return min_distance;
}

int main() {
    Eigen::VectorXd x(3);
    x << 1, 2, 3;
    
    std::list<Eigen::VectorXd> C;
    
    for (int i=0; i<3; i++)
    {
        Eigen::VectorXd temp = Eigen::VectorXd::Random(3);
        C.push_back(temp);
    }
    
    double dist = min_dist(x, C);
    
    std::cout << dist << std::endl;
 
    return 0;
}

In [ ]:
%%bash
g++ junk.cpp -o junk.exe -std=c++14 -I/usr/include/eigen3

In [ ]:
%%bash
./junk.exe

In [ ]:
double min_dist(Eigen::VectorXd x, std::list<Eigen::VectorXd> C)
{
    double min_distance = std::numeric_limits<double>::infinity();
    
    for (Eigen::VectorXd centroid: C)
    {
        double distance = (x-centroid).norm();
        
        if (distance < min_distance)
        {
            min_distance = distance;
        }
    }
    
    return min_distance;
}